## 国勢調査活用セミナー

### 課題: リフォーム会社でターゲットとなる地域をデータを活用して探す
- 国勢調査の位置データを活用して、営業に効果的な地域を探す
- データ分析というよりも、どのようにデータを活用するか
- この後のステップにデータ分析が待っている

### 今回のデータ活用のポイント
- データを活用して営業地域を決めることにより、効率が上がる！
- 効率を上げるためのテストができる




In [1]:
# Important library for many geopython libraries
!apt install gdal-bin python-gdal python3-gdal &> /dev/null
# Install rtree - Geopandas requirment
!apt install python3-rtree  &> /dev/null
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git  &> /dev/null
# Install descartes - Geopandas requirment
!pip install descartes  &> /dev/null
!pip install plotly -U &> /dev/null

In [2]:
import geopandas as gpd
import folium
import shapely
import plotly.express as px
import pandas as pd

## 前もって作られた国勢調査の250メートルメッシュデータを読み込み整形する

- いきなりだと使えないので、整形する。

In [ ]:
# データの読み込みと、CRSの設定
data = gpd.read_file('/content/drive/MyDrive/work/20211017/data/merged_Q5235.geojson')
data = data.set_crs('EPSG:4326', allow_override=True)
data.head(3)

,KEY_CODE,HTKSYORI,HTKSAKI,GASSAN,雇用者（役員を含む）（１５歳以上）総数,雇用者（役員を含む）（１５歳以上）男,雇用者（役員を含む）（１５歳以上）女,正規の職員・従業員（１５歳以上）総数,正規の職員・従業員（１５歳以上）男,正規の職員・従業員（１５歳以上）女,労働者派遣事業所の派遣社員（１５歳以上）総数,労働者派遣事業所の派遣社員（１５歳以上）男,労働者派遣事業所の派遣社員（１５歳以上）女,パート・アルバイト・その他（１５歳以上）総数,パート・アルバイト・その他（１５歳以上）男,パート・アルバイト・その他（１５歳以上）女,自営業主（家庭内職者を含む）（１５歳以上）総数,自営業主（家庭内職者を含む）（１５歳以上）男,自営業主（家庭内職者を含む）（１５歳以上）女,家族従業者（１５歳以上）総数,家族従業者（１５歳以上）男,家族従業者（１５歳以上）女,居住期間出生時から総数,居住期間出生時から男,居住期間出生時から女,居住期間１年未満総数,居住期間１年未満男,居住期間１年未満女,居住期間１～５年未満総数,居住期間１～５年未満男,居住期間１～５年未満女,居住期間５～１０年未満総数,居住期間５～１０年未満男,居住期間５～１０年未満女,居住期間１０～２０年未満総数,居住期間１０～２０年未満男,居住期間１０～２０年未満女,居住期間２０年以上総数,居住期間２０年以上男,居住期間２０年以上女,...,１５～６４歳人口総数,１５～６４歳人口男,１５～６４歳人口女,２０歳以上人口総数,２０歳以上人口男,２０歳以上人口女,６５歳以上人口総数,６５歳以上人口男,６５歳以上人口女,７５歳以上人口総数,７５歳以上人口男,７５歳以上人口女,外国人人口総数,外国人人口男,外国人人口女,世帯総数,一般世帯数,１人世帯の一般世帯数,２人世帯の一般世帯数,３人世帯の一般世帯数,４人世帯の一般世帯数,５人世帯の一般世帯数,６人世帯の一般世帯数,７人以上世帯の一般世帯数,親族のみの一般世帯数,核家族の一般世帯数,核家族以外の一般世帯数,６歳未満世帯員のいる一般世帯数,６５歳以上世帯員のいる一般世帯数,世帯主の年齢が２０～２９歳の１人世帯の一般世帯数,高齢単身の一般世帯数,高齢夫婦のみの一般世帯数,農林漁業就業者世帯の一般世帯数,農林漁業・非農林漁業就業者混合世帯の一般世帯数,非農林漁業就業者世帯の一般世帯数,非就業者世帯の一般世帯数,当地に常住する１５歳以上就業者・通学者総数,当地に常住する１５歳以上就業者・通学者就業者数,当地に常住する１５歳以上就業者・通学者通学者数,geometry
0,5235000011,1.0,NaN,5235000012,13,10,3,11,10,1,0,0,0,2,0,2,1,1,0,0,0,0,3,2,1,4,2,2,6,4,2,11,5,6,3,1,2,5,3,2,...,30,16,14,36,19,17,6,3,3,3,2,1,0,0,0,20,20,12,8,4,1,0,0,0,13,13,0,4,3,5,0,3,0,0,12,11,16,16,0,"POLYGON ((135.00313 34.66667, 135.00000 34.666..."
1,5235000012,2.0,5.235000e+09,None,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,...,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,5,5,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,*,"POLYGON ((135.00625 34.66667, 135.00313 34.666..."
2,5235000013,0.0,NaN,None,30,16,14,18,14,4,1,1,0,9,0,9,4,3,1,0,0,0,11,4,7,0,0,0,0,0,0,49,26,23,9,6,3,10,5,5,...,50,24,26,57,28,29,9,5,4,6,4,2,0,0,0,27,27,2,5,4,12,3,1,0,24,22,2,6,5,0,0,2,0,0,20,7,37,34,3,"POLYGON ((135.00313 34.66875, 135.00000 34.668..."


In [ ]:
data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 35021 entries, 0 to 35020
Data columns (total 89 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   KEY_CODE                  35021 non-null  int64   
 1   HTKSYORI                  35021 non-null  float64 
 2   HTKSAKI                   6458 non-null   float64 
 3   GASSAN                    4511 non-null   object  
 4   雇用者（役員を含む）（１５歳以上）総数       35021 non-null  object  
 5   雇用者（役員を含む）（１５歳以上）男        35021 non-null  object  
 6   雇用者（役員を含む）（１５歳以上）女        35021 non-null  object  
 7   正規の職員・従業員（１５歳以上）総数        35021 non-null  object  
 8   正規の職員・従業員（１５歳以上）男         35021 non-null  object  
 9   正規の職員・従業員（１５歳以上）女         35021 non-null  object  
 10  労働者派遣事業所の派遣社員（１５歳以上）総数    35021 non-null  object  
 11  労働者派遣事業所の派遣社員（１５歳以上）男     35021 non-null  object  
 12  労働者派遣事業所の派遣社員（１５歳以上）女     35021 non-null  object  
 13  パート・アルバイト・その他（１５歳以上）総数    35021 non-nu

- 変なマークが入っており、そこが数値になっていないことが分かる
- それを直して数値データとして活用できるように処理する

In [ ]:
data = data.replace('*', '0')
data.iloc[:, 4:-1] = data.iloc[:, 4: -1].astype(int)
data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 35021 entries, 0 to 35020
Data columns (total 89 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   KEY_CODE                  35021 non-null  int64   
 1   HTKSYORI                  35021 non-null  float64 
 2   HTKSAKI                   6458 non-null   float64 
 3   GASSAN                    4511 non-null   object  
 4   雇用者（役員を含む）（１５歳以上）総数       35021 non-null  int64   
 5   雇用者（役員を含む）（１５歳以上）男        35021 non-null  int64   
 6   雇用者（役員を含む）（１５歳以上）女        35021 non-null  int64   
 7   正規の職員・従業員（１５歳以上）総数        35021 non-null  int64   
 8   正規の職員・従業員（１５歳以上）男         35021 non-null  int64   
 9   正規の職員・従業員（１５歳以上）女         35021 non-null  int64   
 10  労働者派遣事業所の派遣社員（１５歳以上）総数    35021 non-null  int64   
 11  労働者派遣事業所の派遣社員（１５歳以上）男     35021 non-null  int64   
 12  労働者派遣事業所の派遣社員（１５歳以上）女     35021 non-null  int64   
 13  パート・アルバイト・その他（１５歳以上）総数    35021 non-nu

- データを数値化できた
- 案外データを使えるようにするのも手がかかる

## どのようなデータが含まれているかを観察し、使えそうなデータをピックアップする
- データの詳細を確認する
- リフォーム会社のセールスのターゲティングに役立ちそうなファクターをピックアップする
- その後閾値を決める

In [ ]:
# データの内容の確認
data.head(3)

,KEY_CODE,HTKSYORI,HTKSAKI,GASSAN,雇用者（役員を含む）（１５歳以上）総数,雇用者（役員を含む）（１５歳以上）男,雇用者（役員を含む）（１５歳以上）女,正規の職員・従業員（１５歳以上）総数,正規の職員・従業員（１５歳以上）男,正規の職員・従業員（１５歳以上）女,労働者派遣事業所の派遣社員（１５歳以上）総数,労働者派遣事業所の派遣社員（１５歳以上）男,労働者派遣事業所の派遣社員（１５歳以上）女,パート・アルバイト・その他（１５歳以上）総数,パート・アルバイト・その他（１５歳以上）男,パート・アルバイト・その他（１５歳以上）女,自営業主（家庭内職者を含む）（１５歳以上）総数,自営業主（家庭内職者を含む）（１５歳以上）男,自営業主（家庭内職者を含む）（１５歳以上）女,家族従業者（１５歳以上）総数,家族従業者（１５歳以上）男,家族従業者（１５歳以上）女,居住期間出生時から総数,居住期間出生時から男,居住期間出生時から女,居住期間１年未満総数,居住期間１年未満男,居住期間１年未満女,居住期間１～５年未満総数,居住期間１～５年未満男,居住期間１～５年未満女,居住期間５～１０年未満総数,居住期間５～１０年未満男,居住期間５～１０年未満女,居住期間１０～２０年未満総数,居住期間１０～２０年未満男,居住期間１０～２０年未満女,居住期間２０年以上総数,居住期間２０年以上男,居住期間２０年以上女,...,１５～６４歳人口総数,１５～６４歳人口男,１５～６４歳人口女,２０歳以上人口総数,２０歳以上人口男,２０歳以上人口女,６５歳以上人口総数,６５歳以上人口男,６５歳以上人口女,７５歳以上人口総数,７５歳以上人口男,７５歳以上人口女,外国人人口総数,外国人人口男,外国人人口女,世帯総数,一般世帯数,１人世帯の一般世帯数,２人世帯の一般世帯数,３人世帯の一般世帯数,４人世帯の一般世帯数,５人世帯の一般世帯数,６人世帯の一般世帯数,７人以上世帯の一般世帯数,親族のみの一般世帯数,核家族の一般世帯数,核家族以外の一般世帯数,６歳未満世帯員のいる一般世帯数,６５歳以上世帯員のいる一般世帯数,世帯主の年齢が２０～２９歳の１人世帯の一般世帯数,高齢単身の一般世帯数,高齢夫婦のみの一般世帯数,農林漁業就業者世帯の一般世帯数,農林漁業・非農林漁業就業者混合世帯の一般世帯数,非農林漁業就業者世帯の一般世帯数,非就業者世帯の一般世帯数,当地に常住する１５歳以上就業者・通学者総数,当地に常住する１５歳以上就業者・通学者就業者数,当地に常住する１５歳以上就業者・通学者通学者数,geometry
0,5235000011,1.0,NaN,5235000012,13,10,3,11,10,1,0,0,0,2,0,2,1,1,0,0,0,0,3,2,1,4,2,2,6,4,2,11,5,6,3,1,2,5,3,2,...,30,16,14,36,19,17,6,3,3,3,2,1,0,0,0,20,20,12,8,4,1,0,0,0,13,13,0,4,3,5,0,3,0,0,12,11,16,16,0,"POLYGON ((135.00313 34.66667, 135.00000 34.666..."
1,5235000012,2.0,5.235000e+09,None,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"POLYGON ((135.00625 34.66667, 135.00313 34.666..."
2,5235000013,0.0,NaN,None,30,16,14,18,14,4,1,1,0,9,0,9,4,3,1,0,0,0,11,4,7,0,0,0,0,0,0,49,26,23,9,6,3,10,5,5,...,50,24,26,57,28,29,9,5,4,6,4,2,0,0,0,27,27,2,5,4,12,3,1,0,24,22,2,6,5,0,0,2,0,0,20,7,37,34,3,"POLYGON ((135.00313 34.66875, 135.00000 34.668..."


In [ ]:
# foliumを使ってデータのエリアを可視化
# 背景の設定も様々な設定が行える。見易さや目的により変更できる。

center = shapely.geometry.MultiPolygon(data.geometry.values).centroid
area_bounds = shapely.geometry.MultiPolygon(data.geometry.values).bounds
box = shapely.geometry.box(area_bounds[0], area_bounds[1], area_bounds[2], area_bounds[3])
map = folium.Map([center.y, center.x],
                 tiles='https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
                 attr='google map',
                 #tiles='Stamen Toner',
                 zoom_start=8
                 )

folium.GeoJson(box).add_to(map)
map

In [ ]:
selected_data = data[['人口総数', 'KEY_CODE', 'geometry']]
selected_data = selected_data.set_crs('EPSG:4326')


center = shapely.geometry.MultiPolygon(selected_data.geometry.values).centroid

map = folium.Map([center.y, center.x],
                 #tiles='https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
                 #attr='google map',
                 tiles='Stamen Toner',
                 )

folium.Choropleth(geo_data=selected_data, data=selected_data, columns=['KEY_CODE', '人口総数'],
                key_on='feature.properties.KEY_CODE', name='total', fill_color='OrRd',
                line_opacity=0.1).add_to(map)

map

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
data.columns

Index(['KEY_CODE', 'HTKSYORI', 'HTKSAKI', 'GASSAN', '雇用者（役員を含む）（１５歳以上）総数',
       '雇用者（役員を含む）（１５歳以上）男', '雇用者（役員を含む）（１５歳以上）女', '正規の職員・従業員（１５歳以上）総数',
       '正規の職員・従業員（１５歳以上）男', '正規の職員・従業員（１５歳以上）女', '労働者派遣事業所の派遣社員（１５歳以上）総数',
       '労働者派遣事業所の派遣社員（１５歳以上）男', '労働者派遣事業所の派遣社員（１５歳以上）女',
       'パート・アルバイト・その他（１５歳以上）総数', 'パート・アルバイト・その他（１５歳以上）男',
       'パート・アルバイト・その他（１５歳以上）女', '自営業主（家庭内職者を含む）（１５歳以上）総数',
       '自営業主（家庭内職者を含む）（１５歳以上）男', '自営業主（家庭内職者を含む）（１５歳以上）女', '家族従業者（１５歳以上）総数',
       '家族従業者（１５歳以上）男', '家族従業者（１５歳以上）女', '居住期間出生時から総数', '居住期間出生時から男',
       '居住期間出生時から女', '居住期間１年未満総数', '居住期間１年未満男', '居住期間１年未満女', '居住期間１～５年未満総数',
       '居住期間１～５年未満男', '居住期間１～５年未満女', '居住期間５～１０年未満総数', '居住期間５～１０年未満男',
       '居住期間５～１０年未満女', '居住期間１０～２０年未満総数', '居住期間１０～２０年未満男', '居住期間１０～２０年未満女',
       '居住期間２０年以上総数', '居住期間２０年以上男', '居住期間２０年以上女', '人口総数', '人口総数男', '人口総数女',
       '０～１４歳人口総数', '０～１４歳人口男', '０～１４歳人口女', '１５歳以上人口総数', '１５歳以上人口男',
       '１５歳以上人口女', '１５～６４歳人口総数', '１５～６４歳人口男', '１５～６４歳人口女', '２０歳以上人口総数',
       '２０歳以上人口男'

### 営業活動が有利になりそうな要素を選ぶ

- 総数は多い所の方が良さそう
- 居住期間が居住期間５～１０年未満総数、10－20年未満総数 or 20年以上の割合が多い地域が良さそう(どちらが良いかテストするデータとする)
- 子供の教育費がない世帯が良さそう

- ほかにも入れられそうだが、今回は都合3つとします

In [ ]:
# ↑のデータだけのデータセットを作成する

data1 = data[['KEY_CODE', '人口総数', '居住期間５～１０年未満総数','居住期間１０～２０年未満総数','居住期間２０年以上総数', '一般世帯数','２人世帯の一般世帯数', 'geometry']].copy()

In [ ]:
data1.head()

,KEY_CODE,人口総数,居住期間５～１０年未満総数,居住期間１０～２０年未満総数,居住期間２０年以上総数,一般世帯数,２人世帯の一般世帯数,geometry
0,5235000011,33,11,3,5,20,8,"POLYGON ((135.00313 34.66667, 135.00000 34.666..."
1,5235000012,11,0,0,0,5,0,"POLYGON ((135.00625 34.66667, 135.00313 34.666..."
2,5235000013,93,49,9,10,27,5,"POLYGON ((135.00313 34.66875, 135.00000 34.668..."
3,5235000014,278,21,73,48,110,28,"POLYGON ((135.00625 34.66875, 135.00313 34.668..."
4,5235000021,110,25,9,10,44,13,"POLYGON ((135.00938 34.66667, 135.00625 34.666..."


### レシオを作る
- 人口総数はそのままで分かりやすい
- 居住期間は独自に10年以上(over_10)という項目を作成する
- 居住期間は人数なので、人口総数で割ってレシオを作る
- 世帯数

In [ ]:
## 居住期間データの作成

data1['over_10'] = data1['居住期間１０～２０年未満総数'] + data1['居住期間２０年以上総数']
data1['ratio_over10'] = data1['over_10'] / data1['人口総数']
data1['ratio_5to10'] = data1['居住期間５～１０年未満総数'] / data1['人口総数']
data1['ratio_10to20'] = data1['居住期間１０～２０年未満総数'] / data1['人口総数']
data1['ratio_over20'] = data1['居住期間２０年以上総数'] / data1['人口総数']

## 2人世帯の割合の作成

data1['household_ratio'] = data1['２人世帯の一般世帯数'] / data1['一般世帯数']

In [ ]:
# データ確認

data1.head()

,KEY_CODE,人口総数,居住期間５～１０年未満総数,居住期間１０～２０年未満総数,居住期間２０年以上総数,一般世帯数,２人世帯の一般世帯数,geometry,over_10,ratio_over10,ratio_5to10,ratio_10to20,ratio_over20,household_ratio
0,5235000011,33,11,3,5,20,8,"POLYGON ((135.00313 34.66667, 135.00000 34.666...",8,0.242424,0.333333,0.090909,0.151515,0.400000
1,5235000012,11,0,0,0,5,0,"POLYGON ((135.00625 34.66667, 135.00313 34.666...",0,0.000000,0.000000,0.000000,0.000000,0.000000
2,5235000013,93,49,9,10,27,5,"POLYGON ((135.00313 34.66875, 135.00000 34.668...",19,0.204301,0.526882,0.096774,0.107527,0.185185
3,5235000014,278,21,73,48,110,28,"POLYGON ((135.00625 34.66875, 135.00313 34.668...",121,0.435252,0.075540,0.262590,0.172662,0.254545
4,5235000021,110,25,9,10,44,13,"POLYGON ((135.00938 34.66667, 135.00625 34.666...",19,0.172727,0.227273,0.081818,0.090909,0.295455


# ターゲットエリアを定量的に決める

- 人口のボリュームはどの程度？
- 2人世帯の閾値
- 居住期間の閾値

In [ ]:
# 人口のヒストグラム
# 250m×250mの地域にどの程度人がいると自社にとって多い？効率的ととらえられるか？
# 多めにしておく？

px.histogram(data1['人口総数'])

In [ ]:
# 50%の105人にする。
data1['人口総数'].describe()

count    35021.000000
mean       318.012021
std        416.002068
min          1.000000
25%         15.000000
50%        105.000000
75%        514.000000
max       4562.000000
Name: 人口総数, dtype: float64

In [ ]:
# レシオ観察
data1[['over_10', '居住期間５～１０年未満総数', '居住期間１０～２０年未満総数', '居住期間２０年以上総数']].describe()

,over_10,居住期間５～１０年未満総数,居住期間１０～２０年未満総数,居住期間２０年以上総数
count,35021.00000,35021.000000,35021.000000,35021.000000
mean,141.94786,40.265241,60.945204,81.002656
std,186.66181,67.341340,93.677086,109.910528
min,0.00000,0.000000,0.000000,0.000000
25%,8.00000,0.000000,1.000000,5.000000
50%,44.00000,9.000000,14.000000,25.000000
75%,239.00000,60.000000,93.000000,131.000000
max,1994.00000,1821.000000,1578.000000,1446.000000


In [ ]:
# レシオ観察
data1[['ratio_over10', 'ratio_over20', 'ratio_10to20', 'ratio_5to10']].describe()

,ratio_over10,ratio_over20,ratio_10to20,ratio_5to10
count,35021.000000,35021.000000,35021.000000,35021.000000
mean,0.435049,0.286377,0.148671,0.094604
std,0.449078,0.336327,0.215880,0.171803
min,0.000000,0.000000,0.000000,0.000000
25%,0.237762,0.072327,0.004902,0.000000
50%,0.454545,0.265060,0.136364,0.083333
75%,0.589744,0.409677,0.207661,0.129103
max,32.000000,21.500000,24.500000,14.500000


## レシオが1を飛び越えている地域がある
- データがマージされていたりで・・・
- 人口総数を上目に設定すると、大丈夫になる

In [ ]:
#　人口総数が105人以上のデータを作成する
data2 = data1.query('人口総数 > 105')
data2[['ratio_over10', 'ratio_over20', 'ratio_10to20', 'ratio_5to10']].describe()

,ratio_over10,ratio_over20,ratio_10to20,ratio_5to10
count,17491.000000,17491.000000,17491.000000,17491.000000
mean,0.461757,0.270032,0.191725,0.125511
std,0.161417,0.141669,0.101574,0.075761
min,0.000000,0.000000,0.000000,0.000000
25%,0.367226,0.171429,0.132075,0.084058
50%,0.469214,0.264434,0.176265,0.110619
75%,0.571054,0.363832,0.230197,0.145111
max,0.989071,0.967213,0.846561,0.920139


In [ ]:
px.histogram(data2['household_ratio'])

In [ ]:
data2['household_ratio'].describe()

count    17457.000000
mean         0.289644
std          0.084791
min          0.000000
25%          0.237288
50%          0.289720
75%          0.342342
max          1.250000
Name: household_ratio, dtype: float64

## 条件を決定する

- 人口総数は105人以上
- ratio: 5-10の閾値は0.11(a)
- ratio: 10－20の閾値は0.23(b)
- 2人世帯は0.34以上

In [ ]:
data_a = data2.query('ratio_5to10 > 0.11 & household_ratio > 0.34').copy()
data_b = data2.query('ratio_10to20 > 0.23 & household_ratio > 0.34').copy()


In [ ]:
data_a.describe()

,KEY_CODE,人口総数,居住期間５～１０年未満総数,居住期間１０～２０年未満総数,居住期間２０年以上総数,一般世帯数,２人世帯の一般世帯数,over_10,ratio_over10,ratio_5to10,ratio_10to20,ratio_over20,household_ratio
count,1.826000e+03,1826.000000,1826.000000,1826.000000,1826.000000,1826.000000,1826.000000,1826.000000,1826.000000,1826.000000,1826.000000,1826.000000,1826.000000
mean,5.235185e+09,519.248631,76.657722,101.736035,171.415115,213.018072,80.403067,273.151150,0.533444,0.151771,0.195096,0.338348,0.387568
std,1.523331e+05,358.287442,57.511533,79.180601,122.133319,160.764544,58.936582,185.522026,0.124855,0.050997,0.078588,0.120447,0.054104
min,5.235001e+09,106.000000,12.000000,0.000000,0.000000,4.000000,2.000000,2.000000,0.005917,0.110016,0.000000,0.000000,0.340067
25%,5.235062e+09,256.250000,38.000000,46.000000,82.000000,99.000000,39.000000,136.000000,0.469525,0.121656,0.149386,0.265823,0.355189
50%,5.235144e+09,433.500000,62.000000,82.000000,148.000000,172.000000,66.000000,238.000000,0.550625,0.136700,0.185390,0.345989,0.375000
75%,5.235247e+09,667.750000,97.000000,138.000000,232.000000,277.750000,103.000000,358.000000,0.618398,0.164577,0.228571,0.420781,0.404762
max,5.235780e+09,2431.000000,508.000000,568.000000,857.000000,1282.000000,449.000000,1232.000000,0.838710,0.692982,0.761905,0.661290,1.250000


In [ ]:
data_b.describe()

,KEY_CODE,人口総数,居住期間５～１０年未満総数,居住期間１０～２０年未満総数,居住期間２０年以上総数,一般世帯数,２人世帯の一般世帯数,over_10,ratio_over10,ratio_5to10,ratio_10to20,ratio_over20,household_ratio
count,1.071000e+03,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000,1071.000000
mean,5.235185e+09,462.357610,51.708683,135.745098,146.484594,183.820728,69.189542,282.229692,0.626906,0.107868,0.299571,0.327336,0.384630
std,1.506214e+05,310.270184,45.668402,93.035303,104.147000,133.274664,48.076735,177.402645,0.102673,0.046394,0.085404,0.121039,0.042201
min,5.235001e+09,106.000000,1.000000,25.000000,0.000000,17.000000,6.000000,37.000000,0.258322,0.008403,0.230016,0.000000,0.340136
25%,5.235064e+09,235.000000,22.000000,70.000000,67.000000,90.500000,36.000000,148.500000,0.562866,0.078693,0.245427,0.255352,0.354613
50%,5.235147e+09,407.000000,39.000000,112.000000,125.000000,156.000000,60.000000,252.000000,0.632051,0.101617,0.269531,0.342776,0.373333
75%,5.235245e+09,581.000000,67.000000,173.000000,202.000000,226.500000,85.000000,363.500000,0.699144,0.130652,0.316465,0.411807,0.403671
max,5.235780e+09,2203.000000,498.000000,740.000000,808.000000,994.000000,351.000000,1381.000000,0.855263,0.417910,0.785714,0.613793,0.886364


In [ ]:
data_a['label'] = 'a'
data_b['label'] = 'b'
merged = pd.concat([data_a, data_b])

In [ ]:
merged

,KEY_CODE,人口総数,居住期間５～１０年未満総数,居住期間１０～２０年未満総数,居住期間２０年以上総数,一般世帯数,２人世帯の一般世帯数,geometry,over_10,ratio_over10,ratio_5to10,ratio_10to20,ratio_over20,household_ratio,label
92,5235000623,536,59,62,228,289,108,"POLYGON ((135.08437 34.66875, 135.08125 34.668...",290,0.541045,0.110075,0.115672,0.425373,0.373702,a
101,5235000644,1072,144,184,412,524,193,"POLYGON ((135.08750 34.67292, 135.08437 34.672...",596,0.555970,0.134328,0.171642,0.384328,0.368321,a
112,5235000733,806,96,158,260,352,123,"POLYGON ((135.09062 34.67292, 135.08750 34.672...",418,0.518610,0.119107,0.196030,0.322581,0.349432,a
116,5235000743,619,119,86,274,234,92,"POLYGON ((135.09688 34.67292, 135.09375 34.672...",360,0.581583,0.192246,0.138934,0.442649,0.393162,a
117,5235000744,504,70,96,233,202,87,"POLYGON ((135.10000 34.67292, 135.09688 34.672...",329,0.652778,0.138889,0.190476,0.462302,0.430693,a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34205,5235726031,410,80,110,103,168,61,"POLYGON ((135.25313 35.30417, 135.25000 35.304...",213,0.519512,0.195122,0.268293,0.251220,0.363095,b
34216,5235726121,163,16,39,76,60,25,"POLYGON ((135.27187 35.30000, 135.26875 35.300...",115,0.705521,0.098160,0.239264,0.466258,0.416667,b
34981,5235777914,277,45,113,83,111,50,"POLYGON ((135.99375 35.31042, 135.99062 35.310...",196,0.707581,0.162455,0.407942,0.299639,0.450450,b
34982,5235777923,136,16,41,62,57,25,"POLYGON ((135.99687 35.31042, 135.99375 35.310...",103,0.757353,0.117647,0.301471,0.455882,0.438596,b


In [ ]:
## データの被りの確認
## 両方の条件を満たすところもある: KEY_CODEが2だと、両方を満たしている
## ABテストのためにはかぶりのない所を使いたい

merged['KEY_CODE'].value_counts()

5235130143    2
5235242514    2
5235055924    2
5235137842    2
5235154341    2
             ..
5235059234    1
5235012131    1
5235003941    1
5235157542    1
5235159042    1
Name: KEY_CODE, Length: 2452, dtype: int64

In [ ]:
code_count = pd.DataFrame(merged['KEY_CODE'].value_counts()).reset_index()
code_count = code_count.query('KEY_CODE == 1')
code_count = code_count.rename({'index': 'code'}, axis=1)

In [ ]:
code_count

,code,KEY_CODE
445,5235152112,1
446,5235137114,1
447,5235154232,1
448,5235145212,1
449,5235005134,1
...,...,...
2447,5235059234,1
2448,5235012131,1
2449,5235003941,1
2450,5235157542,1


In [ ]:
merged2 = merged[merged['KEY_CODE'].isin(code_count['code'].values)]
print(merged2.shape)
merged2['label'].value_counts()

(2007, 15)


a    1381
b     626
Name: label, dtype: int64

In [ ]:
merged = merged.set_crs('EPSG:4326')
center = shapely.geometry.MultiPolygon(merged.geometry.values).centroid

a = merged.query('label == "a"')
b = merged.query('label == "b"')

map = folium.Map([center.y, center.x],
                 #tiles='https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
                 #attr='google map'
                 tiles='Stamen Toner'
                 )

folium.Choropleth(geo_data=merged, data=merged, columns=['KEY_CODE', '人口総数'],
                key_on='feature.properties.KEY_CODE', fill_color='OrRd', name='total').add_to(map)

folium.GeoJson(
    a, 
    tooltip=folium.features.GeoJsonTooltip(fields=["KEY_CODE", "人口総数", "household_ratio"]),
    style_function = lambda x: {'color': 'red'},
    name='red_labels'

).add_to(map)


folium.GeoJson(
    b, 
    tooltip=folium.features.GeoJsonTooltip(fields=["KEY_CODE", "人口総数", "household_ratio"]),
    style_function = lambda x: {'color': 'blue'},
    name='blue_labels'

).add_to(map)


folium.LayerControl().add_to(map)
map

Output hidden; open in https://colab.research.google.com to view.

### エリアのピックアップ作業とその後

- この中から、効率などを考え、エリアを選択する
- 青、赤どちらが良いか、効果測定をする
- それを何度か繰り返す
- 効きそうなファクターなどを検証する（検証: データ分析の段階）
- 他のファクターを試したりする